## Lemmatisation et mots/lemmes les plus fréquents du corpus

In [7]:
!python -m spacy download fr_core_news_md ## chargement du modèle en français de spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 4.6 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [ ]:
import pandas as pd
import spacy
from collections import Counter
import re
from typing import List, Dict, Tuple

def load_and_prepare_data(file_path: str) -> pd.DataFrame:
    """Charge le fichier CSV et prépare les données."""
    df = pd.read_csv(file_path, sep=';')
    
    # Vérifier que les colonnes existent
    required_columns = ['texte', 'discours']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Colonnes manquantes: {missing_columns}")
    
    # Remplacer les valeurs NaN par des chaînes vides
    df['texte'] = df['texte'].fillna('')
    df['discours'] = df['discours'].fillna('')
    
    return df

def preprocess_text(text: str) -> str:
    """Nettoie et préprocesse le texte."""
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer la ponctuation et les caractères spéciaux (garder seulement lettres, chiffres, espaces)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def get_french_stopwords(stopwords_file_path: str = 'stopwords.txt') -> set:
    """
    Retourne un ensemble de mots-outils français à partir d'un fichier,
    augmenté avec des chaînes spécifiques à exclure.
    """
    stopwords = set()
    
    # Charger les stopwords depuis le fichier
    try:
        with open(stopwords_file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Nettoyer chaque ligne (enlever espaces et retours à la ligne)
                word = line.strip().lower()
                if word:  # Ignorer les lignes vides
                    stopwords.add(word)
        print(f"Stopwords chargés: {len(stopwords)} mots")
    except FileNotFoundError:
        print(f"Attention: Le fichier {stopwords_file_path} n'a pas été trouvé.")
        print("Utilisation d'un ensemble vide de stopwords.")
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        print("Utilisation d'un ensemble vide de stopwords.")
    
    # Ajouter les chaînes spécifiques à exclure (en minuscules pour la cohérence)
    custom_stopwords = {
        'sciences po / fonds cevipof',
        'page 1/1',
        'page 1/2', 
        'page 2/2'
    }
    
    return stopwords.union(custom_stopwords)

def analyze_words(text_series: pd.Series, column_name: str, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """Analyse les occurrences de mots dans une série de textes."""
    stopwords = get_french_stopwords(stopwords_file_path)
    word_counter = Counter()
    
    for text in text_series:
        if pd.isna(text) or text == '':
            continue
            
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Diviser en mots
        words = clean_text.split()
        
        # Filtrer les mots-outils et les mots trop courts
        filtered_words = [
            word for word in words 
            if len(word) > 2 and word not in stopwords and word.isalpha()
        ]
        
        word_counter.update(filtered_words)
    
    return dict(word_counter)

def analyze_lemmas(text_series: pd.Series, column_name: str, nlp, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """Analyse les occurrences de lemmes dans une série de textes."""
    stopwords = get_french_stopwords(stopwords_file_path)
    lemma_counter = Counter()
    
    print(f"Analyse des lemmes pour la colonne '{column_name}'...")
    
    for i, text in enumerate(text_series):
        if i % 100 == 0:
            print(f"  Traitement: {i+1}/{len(text_series)} textes")
            
        if pd.isna(text) or text == '':
            continue
            
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Traitement avec spaCy (par chunks pour éviter les limites de mémoire)
        max_length = 1000000  # Limite de spaCy
        if len(clean_text) > max_length:
            chunks = [clean_text[i:i+max_length] for i in range(0, len(clean_text), max_length)]
        else:
            chunks = [clean_text]
        
        for chunk in chunks:
            try:
                doc = nlp(chunk)
                
                # Extraire les lemmes
                lemmas = [
                    token.lemma_.lower() for token in doc 
                    if (not token.is_stop and 
                        not token.is_punct and 
                        not token.is_space and 
                        len(token.lemma_) > 2 and
                        token.lemma_.lower() not in stopwords and
                        token.lemma_.isalpha())
                ]
                
                lemma_counter.update(lemmas)
                
            except Exception as e:
                print(f"Erreur lors du traitement d'un chunk: {e}")
                continue
    
    return dict(lemma_counter)

def count_lemmas_in_series(text_series: pd.Series, column_name: str, nlp, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """
    Compte le nombre total de lemmes dans une série de textes.
    """
    stopwords = get_french_stopwords(stopwords_file_path)
    total_lemmas = 0
    total_unique_lemmas = set()
    processed_texts = 0
    
    print(f"Comptage des lemmes pour la colonne '{column_name}'...")
    
    for i, text in enumerate(text_series):
        if i % 100 == 0:
            print(f"  Comptage: {i+1}/{len(text_series)} textes")
            
        if pd.isna(text) or text == '':
            continue
            
        processed_texts += 1
        
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Traitement avec spaCy (par chunks pour éviter les limites de mémoire)
        max_length = 1000000  # Limite de spaCy
        if len(clean_text) > max_length:
            chunks = [clean_text[i:i+max_length] for i in range(0, len(clean_text), max_length)]
        else:
            chunks = [clean_text]
        
        for chunk in chunks:
            try:
                doc = nlp(chunk)
                
                # Compter et collecter les lemmes
                for token in doc:
                    if (not token.is_stop and 
                        not token.is_punct and 
                        not token.is_space and 
                        len(token.lemma_) > 2 and
                        token.lemma_.lower() not in stopwords and
                        token.lemma_.isalpha()):
                        
                        total_lemmas += 1
                        total_unique_lemmas.add(token.lemma_.lower())
                        
            except Exception as e:
                print(f"Erreur lors du traitement d'un chunk: {e}")
                continue
    
    return {
        'total_lemmas': total_lemmas,
        'unique_lemmas': len(total_unique_lemmas),
        'processed_texts': processed_texts,
        'average_lemmas_per_text': total_lemmas / processed_texts if processed_texts > 0 else 0
    }

def display_top_results(results: Dict[str, int], title: str, top_n: int = 30):
    """Affiche les top résultats."""
    print(f"\n{title}")
    print("=" * len(title))
    
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    
    for i, (item, count) in enumerate(sorted_results[:top_n], 1):
        print(f"{i:2d}. {item:<20} : {count:>6} occurrences")

def display_lemma_stats(stats: Dict[str, int], column_name: str):
    """Affiche les statistiques de comptage des lemmes."""
    print(f"\n{'='*50}")
    print(f"STATISTIQUES DES LEMMES - Colonne '{column_name}'")
    print(f"{'='*50}")
    print(f"Textes traités           : {stats['processed_texts']:>10}")
    print(f"Total des lemmes         : {stats['total_lemmas']:>10}")
    print(f"Lemmes uniques           : {stats['unique_lemmas']:>10}")
    print(f"Moyenne lemmes/texte     : {stats['average_lemmas_per_text']:>10.2f}")
    print(f"{'='*50}")

def main():
    # Configuration
    file_path = "/Users/charlielezin/Desktop/Candidatures58-81-290825.csv"
    stopwords_file_path = "/Users/charlielezin/Desktop/stopwords.txt"
    
    try:
        # Charger spaCy pour le français
        print("Chargement du modèle spaCy français...")
        try:
            nlp = spacy.load("fr_core_news_md")
        except OSError:
            print("Erreur: Le modèle spaCy français n'est pas installé.")
            print("Installez-le avec: python -m spacy download fr_core_news_md")
            return
        
        # Charger les données
        print(f"Chargement du fichier {file_path}...")
        df = load_and_prepare_data(file_path)
        print(f"Fichier chargé: {len(df)} lignes")
        
        # Comptage des lemmes pour chaque colonne
        print("\n" + "="*60)
        print("COMPTAGE DES LEMMES")
        print("="*60)
        
        for column in ['texte', 'discours']:
            lemma_stats = count_lemmas_in_series(df[column], column, nlp, stopwords_file_path)
            display_lemma_stats(lemma_stats, column)
        
        # Analyser les mots pour chaque colonne
        print("\n" + "="*60)
        print("ANALYSE DES MOTS")
        print("="*60)
        
        for column in ['texte', 'discours']:
            print(f"\nAnalyse des mots dans la colonne '{column}'...")
            word_results = analyze_words(df[column], column, stopwords_file_path)
            display_top_results(word_results, f"TOP 30 - MOTS - Colonne '{column}'", 30)
        
        # Analyser les lemmes pour chaque colonne
        print("\n" + "="*60)
        print("ANALYSE DES LEMMES")
        print("="*60)
        
        for column in ['texte', 'discours']:
            print(f"\nAnalyse des lemmes dans la colonne '{column}'...")
            lemma_results = analyze_lemmas(df[column], column, nlp, stopwords_file_path)
            display_top_results(lemma_results, f"TOP 30 - LEMMES - Colonne '{column}'", 30)
        
        
    except FileNotFoundError:
        print(f"Erreur: Le fichier '{file_path}' est introuvable.")
        print("Vérifiez que le fichier est dans le même répertoire que le script.")
    except Exception as e:
        print(f"Erreur: {e}")

if __name__ == "__main__":
    main()

Chargement du modèle spaCy français...
Chargement du fichier /Users/charlielezin/Desktop/Candidatures58-81-290825.csv...
Fichier chargé: 303 lignes

COMPTAGE DES LEMMES
Stopwords chargés: 707 mots
Comptage des lemmes pour la colonne 'texte'...
  Comptage: 1/303 textes
  Comptage: 101/303 textes
  Comptage: 201/303 textes
  Comptage: 301/303 textes

STATISTIQUES DES LEMMES - Colonne 'texte'
Textes traités           :        303
Total des lemmes         :     103628
Lemmes uniques           :       6999
Moyenne lemmes/texte     :     342.01
Stopwords chargés: 707 mots
Comptage des lemmes pour la colonne 'discours'...
  Comptage: 1/303 textes
  Comptage: 101/303 textes
  Comptage: 201/303 textes
  Comptage: 301/303 textes

STATISTIQUES DES LEMMES - Colonne 'discours'
Textes traités           :        303
Total des lemmes         :      89452
Lemmes uniques           :       6422
Moyenne lemmes/texte     :     295.22

ANALYSE DES MOTS

Analyse des mots dans la colonne 'texte'...
Stopwords 

In [ ]:
#!/usr/bin/env python3
"""
Script pour afficher 50 lemmes au hasard avec leurs mots associés
Basé sur l'analyse existante des discours politiques
"""

import pandas as pd
import spacy
import random
from collections import defaultdict
import re

class LemmaWordMapper:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-290825.csv'):
        self.filepath = filepath
        self.df = None
        self.nlp = None
        self.lemma_to_words = defaultdict(set)
        
    def load_data(self):
        """Charge les données des discours"""
        print("Chargement des données...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        # Filtrer pour les tours 1 et 2 avec discours
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de discours chargés : {len(self.df)}")
        return self.df
    
    def load_spacy_model(self):
        """Charge le modèle spaCy français"""
        print("Chargement du modèle spaCy français...")
        try:
            self.nlp = spacy.load("fr_core_news_md")
            print("Modèle spaCy chargé avec succès")
        except OSError:
            print("Erreur: Le modèle spaCy français n'est pas installé.")
            print("Installez-le avec: python -m spacy download fr_core_news_md")
            return False
        return True
    
    def preprocess_text(self, text):
        """Nettoie le texte"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\d+', '', text)
        return text.strip()
    
    def get_french_stopwords(self, stopwords_file_path='/Users/charlielezin/Desktop/stopwords.txt'):
        """Charge les mots vides français depuis un fichier"""
        stopwords = set()
        
        try:
            with open(stopwords_file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    word = line.strip().lower()
                    if word:
                        stopwords.add(word)
            print(f"Stopwords chargés depuis {stopwords_file_path}: {len(stopwords)} mots")
        except FileNotFoundError:
            print(f"Attention: Le fichier {stopwords_file_path} n'a pas été trouvé.")
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier stopwords: {e}")
        
        return stopwords
    
    def extract_lemma_word_mapping(self):
        """Extrait le mapping lemme -> mots pour tous les discours"""
        print("Extraction des lemmes et mots associés...")
        
        stopwords = self.get_french_stopwords()
        
        for i, row in self.df.iterrows():
            if (i + 1) % 50 == 0:
                print(f"  Traitement: {i+1}/{len(self.df)} discours")
            
            text = self.preprocess_text(row['discours'])
            if not text:
                continue
            
            # Traiter par chunks pour éviter les limites de spaCy
            max_length = 500000
            if len(text) > max_length:
                chunks = [text[j:j+max_length] for j in range(0, len(text), max_length)]
            else:
                chunks = [text]
            
            for chunk in chunks:
                try:
                    doc = self.nlp(chunk)
                    
                    for token in doc:
                        if (not token.is_stop and 
                            not token.is_punct and 
                            not token.is_space and 
                            len(token.text) > 2 and
                            len(token.lemma_) > 2 and
                            token.text.lower() not in stopwords and
                            token.lemma_.lower() not in stopwords and
                            token.text.isalpha() and
                            token.lemma_.isalpha()):
                            
                            lemma = token.lemma_.lower()
                            word = token.text.lower()
                            
                            # Ajouter seulement si le mot est différent du lemme
                            if word != lemma:
                                self.lemma_to_words[lemma].add(word)
                
                except Exception as e:
                    print(f"Erreur lors du traitement d'un chunk: {e}")
                    continue
        
        print(f"Extraction terminée. {len(self.lemma_to_words)} lemmes collectés")
    
    def display_random_lemmas(self, n=50):
        """Affiche n lemmes au hasard avec leurs mots associés"""
        print(f"AFFICHAGE DE {n} LEMMES ALÉATOIRES AVEC LEURS MOTS")
        print("="*60)
        
        # Filtrer les lemmes qui ont au moins un mot associé différent
        valid_lemmas = {lemma: words for lemma, words in self.lemma_to_words.items() if len(words) > 0}
        
        if len(valid_lemmas) == 0:
            print("Aucun lemme avec des mots associés trouvé")
            return
        
        # Sélectionner aléatoirement n lemmes
        selected_lemmas = random.sample(list(valid_lemmas.keys()), min(n, len(valid_lemmas)))
        
        for i, lemma in enumerate(selected_lemmas, 1):
            associated_words = sorted(list(valid_lemmas[lemma]))
            words_str = ", ".join(associated_words)
            
            print(f"{i:2d}. {lemma:<15} -> {words_str}")
        
        print(f"\nTotal de lemmes disponibles : {len(valid_lemmas)}")
        print(f"Lemmes affichés : {len(selected_lemmas)}")

def main():
    """Analyse principale"""
    print("ANALYSE LEMMES-MOTS - DISCOURS POLITIQUES")
    print("="*50)
    
    try:
        mapper = LemmaWordMapper()
        
        # Chargement du modèle spaCy
        if not mapper.load_spacy_model():
            return
        
        # Chargement des données
        mapper.load_data()
        
        if len(mapper.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # Extraction du mapping lemmes-mots
        mapper.extract_lemma_word_mapping()
        
        # Affichage aléatoire
        mapper.display_random_lemmas(50)
    
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

## Modèle large

In [16]:
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 3.9 MB/s eta 0:00:0000:0100:04
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [21]:
import pandas as pd
import spacy
from collections import Counter
import re
from typing import List, Dict, Tuple

def load_and_prepare_data(file_path: str) -> pd.DataFrame:
    """Charge le fichier CSV et prépare les données."""
    df = pd.read_csv(file_path, sep=';')
    
    # Vérifier que les colonnes existent
    required_columns = ['texte', 'discours']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Colonnes manquantes: {missing_columns}")
    
    # Remplacer les valeurs NaN par des chaînes vides
    df['texte'] = df['texte'].fillna('')
    df['discours'] = df['discours'].fillna('')
    
    return df

def preprocess_text(text: str) -> str:
    """Nettoie et préprocesse le texte."""
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer la ponctuation et les caractères spéciaux (garder seulement lettres, chiffres, espaces)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def get_french_stopwords(stopwords_file_path: str = 'stopwords.txt') -> set:
    """
    Retourne un ensemble de mots-outils français à partir d'un fichier,
    augmenté avec des chaînes spécifiques à exclure.
    """
    stopwords = set()
    
    # Charger les stopwords depuis le fichier
    try:
        with open(stopwords_file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Nettoyer chaque ligne (enlever espaces et retours à la ligne)
                word = line.strip().lower()
                if word:  # Ignorer les lignes vides
                    stopwords.add(word)
        print(f"Stopwords chargés: {len(stopwords)} mots")
    except FileNotFoundError:
        print(f"Attention: Le fichier {stopwords_file_path} n'a pas été trouvé.")
        print("Utilisation d'un ensemble vide de stopwords.")
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        print("Utilisation d'un ensemble vide de stopwords.")
    
    # Ajouter les chaînes spécifiques à exclure (en minuscules pour la cohérence)
    custom_stopwords = {
        'sciences po / fonds cevipof',
        'page 1/1',
        'page 1/2', 
        'page 2/2'
    }
    
    return stopwords.union(custom_stopwords)

def analyze_words(text_series: pd.Series, column_name: str, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """Analyse les occurrences de mots dans une série de textes."""
    stopwords = get_french_stopwords(stopwords_file_path)
    word_counter = Counter()
    
    for text in text_series:
        if pd.isna(text) or text == '':
            continue
            
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Diviser en mots
        words = clean_text.split()
        
        # Filtrer les mots-outils et les mots trop courts
        filtered_words = [
            word for word in words 
            if len(word) > 2 and word not in stopwords and word.isalpha()
        ]
        
        word_counter.update(filtered_words)
    
    return dict(word_counter)

def analyze_lemmas(text_series: pd.Series, column_name: str, nlp, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """Analyse les occurrences de lemmes dans une série de textes."""
    stopwords = get_french_stopwords(stopwords_file_path)
    lemma_counter = Counter()
    
    print(f"Analyse des lemmes pour la colonne '{column_name}'...")
    
    for i, text in enumerate(text_series):
        if i % 100 == 0:
            print(f"  Traitement: {i+1}/{len(text_series)} textes")
            
        if pd.isna(text) or text == '':
            continue
            
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Traitement avec spaCy (par chunks pour éviter les limites de mémoire)
        max_length = 1000000  # Limite de spaCy
        if len(clean_text) > max_length:
            chunks = [clean_text[i:i+max_length] for i in range(0, len(clean_text), max_length)]
        else:
            chunks = [clean_text]
        
        for chunk in chunks:
            try:
                doc = nlp(chunk)
                
                # Extraire les lemmes
                lemmas = [
                    token.lemma_.lower() for token in doc 
                    if (not token.is_stop and 
                        not token.is_punct and 
                        not token.is_space and 
                        len(token.lemma_) > 2 and
                        token.lemma_.lower() not in stopwords and
                        token.lemma_.isalpha())
                ]
                
                lemma_counter.update(lemmas)
                
            except Exception as e:
                print(f"Erreur lors du traitement d'un chunk: {e}")
                continue
    
    return dict(lemma_counter)

def count_lemmas_in_series(text_series: pd.Series, column_name: str, nlp, stopwords_file_path: str = 'stopwords.txt') -> Dict[str, int]:
    """
    Compte le nombre total de lemmes dans une série de textes.
    """
    stopwords = get_french_stopwords(stopwords_file_path)
    total_lemmas = 0
    total_unique_lemmas = set()
    processed_texts = 0
    
    print(f"Comptage des lemmes pour la colonne '{column_name}'...")
    
    for i, text in enumerate(text_series):
        if i % 100 == 0:
            print(f"  Comptage: {i+1}/{len(text_series)} textes")
            
        if pd.isna(text) or text == '':
            continue
            
        processed_texts += 1
        
        # Préprocesser le texte
        clean_text = preprocess_text(str(text))
        
        # Traitement avec spaCy (par chunks pour éviter les limites de mémoire)
        max_length = 1000000  # Limite de spaCy
        if len(clean_text) > max_length:
            chunks = [clean_text[i:i+max_length] for i in range(0, len(clean_text), max_length)]
        else:
            chunks = [clean_text]
        
        for chunk in chunks:
            try:
                doc = nlp(chunk)
                
                # Compter et collecter les lemmes
                for token in doc:
                    if (not token.is_stop and 
                        not token.is_punct and 
                        not token.is_space and 
                        len(token.lemma_) > 2 and
                        token.lemma_.lower() not in stopwords and
                        token.lemma_.isalpha()):
                        
                        total_lemmas += 1
                        total_unique_lemmas.add(token.lemma_.lower())
                        
            except Exception as e:
                print(f"Erreur lors du traitement d'un chunk: {e}")
                continue
    
    return {
        'total_lemmas': total_lemmas,
        'unique_lemmas': len(total_unique_lemmas),
        'processed_texts': processed_texts,
        'average_lemmas_per_text': total_lemmas / processed_texts if processed_texts > 0 else 0
    }

def display_top_results(results: Dict[str, int], title: str, top_n: int = 30):
    """Affiche les top résultats."""
    print(f"\n{title}")
    print("=" * len(title))
    
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    
    for i, (item, count) in enumerate(sorted_results[:top_n], 1):
        print(f"{i:2d}. {item:<20} : {count:>6} occurrences")

def display_lemma_stats(stats: Dict[str, int], column_name: str):
    """Affiche les statistiques de comptage des lemmes."""
    print(f"\n{'='*50}")
    print(f"STATISTIQUES DES LEMMES - Colonne '{column_name}'")
    print(f"{'='*50}")
    print(f"Textes traités           : {stats['processed_texts']:>10}")
    print(f"Total des lemmes         : {stats['total_lemmas']:>10}")
    print(f"Lemmes uniques           : {stats['unique_lemmas']:>10}")
    print(f"Moyenne lemmes/texte     : {stats['average_lemmas_per_text']:>10.2f}")
    print(f"{'='*50}")

def main():
    # Configuration
    file_path = "/Users/charlielezin/Desktop/Candidatures58-81-290825.csv"
    stopwords_file_path = "/Users/charlielezin/Desktop/stopwords.txt"
    
    try:
        # Charger spaCy pour le français
        print("Chargement du modèle spaCy français...")
        try:
            nlp = spacy.load("fr_core_news_lg")
        except OSError:
            print("Erreur: Le modèle spaCy français n'est pas installé.")
            print("Installez-le avec: python -m spacy download fr_core_news_lg")
            return
        
        # Charger les données
        print(f"Chargement du fichier {file_path}...")
        df = load_and_prepare_data(file_path)
        print(f"Fichier chargé: {len(df)} lignes")
        
        # Comptage des lemmes pour chaque colonne
        print("\n" + "="*60)
        print("COMPTAGE DES LEMMES")
        print("="*60)
        
        for column in ['texte', 'discours']:
            lemma_stats = count_lemmas_in_series(df[column], column, nlp, stopwords_file_path)
            display_lemma_stats(lemma_stats, column)
        
        # Analyser les mots pour chaque colonne
        print("\n" + "="*60)
        print("ANALYSE DES MOTS")
        print("="*60)
        
        for column in ['texte', 'discours']:
            print(f"\nAnalyse des mots dans la colonne '{column}'...")
            word_results = analyze_words(df[column], column, stopwords_file_path)
            display_top_results(word_results, f"TOP 30 - MOTS - Colonne '{column}'", 30)
        
        # Analyser les lemmes pour chaque colonne
        print("\n" + "="*60)
        print("ANALYSE DES LEMMES")
        print("="*60)
        
        for column in ['texte', 'discours']:
            print(f"\nAnalyse des lemmes dans la colonne '{column}'...")
            lemma_results = analyze_lemmas(df[column], column, nlp, stopwords_file_path)
            display_top_results(lemma_results, f"TOP 30 - LEMMES - Colonne '{column}'", 30)
        
        
    except FileNotFoundError:
        print(f"Erreur: Le fichier '{file_path}' est introuvable.")
        print("Vérifiez que le fichier est dans le même répertoire que le script.")
    except Exception as e:
        print(f"Erreur: {e}")

if __name__ == "__main__":
    main()

Chargement du modèle spaCy français...
Chargement du fichier /Users/charlielezin/Desktop/Candidatures58-81-290825.csv...
Fichier chargé: 303 lignes

COMPTAGE DES LEMMES
Stopwords chargés: 707 mots
Comptage des lemmes pour la colonne 'texte'...
  Comptage: 1/303 textes
  Comptage: 101/303 textes
  Comptage: 201/303 textes
  Comptage: 301/303 textes

STATISTIQUES DES LEMMES - Colonne 'texte'
Textes traités           :        303
Total des lemmes         :     103629
Lemmes uniques           :       6892
Moyenne lemmes/texte     :     342.01
Stopwords chargés: 707 mots
Comptage des lemmes pour la colonne 'discours'...
  Comptage: 1/303 textes
  Comptage: 101/303 textes
  Comptage: 201/303 textes
  Comptage: 301/303 textes

STATISTIQUES DES LEMMES - Colonne 'discours'
Textes traités           :        303
Total des lemmes         :      89455
Lemmes uniques           :       6317
Moyenne lemmes/texte     :     295.23

ANALYSE DES MOTS

Analyse des mots dans la colonne 'texte'...
Stopwords 

In [22]:
#!/usr/bin/env python3
"""
Script pour afficher 50 lemmes au hasard avec leurs mots associés
"""

import pandas as pd
import spacy
import random
from collections import defaultdict
import re

class LemmaWordMapper:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-290825.csv'):
        self.filepath = filepath
        self.df = None
        self.nlp = None
        self.lemma_to_words = defaultdict(set)
        
    def load_data(self):
        """Charge les données des discours"""
        print("Chargement des données...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        # Filtrer pour les tours 1 et 2 avec discours
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de discours chargés : {len(self.df)}")
        return self.df
    
    def load_spacy_model(self):
        """Charge le modèle spaCy français"""
        print("Chargement du modèle spaCy français...")
        try:
            self.nlp = spacy.load("fr_core_news_lg")
            print("Modèle spaCy chargé avec succès")
        except OSError:
            print("Erreur: Le modèle spaCy français n'est pas installé.")
            print("Installez-le avec: python -m spacy download fr_core_news_lg")
            return False
        return True
    
    def preprocess_text(self, text):
        """Nettoie le texte"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\d+', '', text)
        return text.strip()
    
    def get_french_stopwords(self, stopwords_file_path='/Users/charlielezin/Desktop/stopwords.txt'):
        """Charge les mots vides français depuis un fichier"""
        stopwords = set()
        
        try:
            with open(stopwords_file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    word = line.strip().lower()
                    if word:
                        stopwords.add(word)
            print(f"Stopwords chargés depuis {stopwords_file_path}: {len(stopwords)} mots")
        except FileNotFoundError:
            print(f"Attention: Le fichier {stopwords_file_path} n'a pas été trouvé.")
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier stopwords: {e}")
        
        return stopwords
    
    def extract_lemma_word_mapping(self):
        """Extrait le mapping lemme -> mots pour tous les discours"""
        print("Extraction des lemmes et mots associés...")
        
        stopwords = self.get_french_stopwords()
        
        for i, row in self.df.iterrows():
            if (i + 1) % 50 == 0:
                print(f"  Traitement: {i+1}/{len(self.df)} discours")
            
            text = self.preprocess_text(row['discours'])
            if not text:
                continue
            
            # Traiter par chunks pour éviter les limites de spaCy
            max_length = 500000
            if len(text) > max_length:
                chunks = [text[j:j+max_length] for j in range(0, len(text), max_length)]
            else:
                chunks = [text]
            
            for chunk in chunks:
                try:
                    doc = self.nlp(chunk)
                    
                    for token in doc:
                        if (not token.is_stop and 
                            not token.is_punct and 
                            not token.is_space and 
                            len(token.text) > 2 and
                            len(token.lemma_) > 2 and
                            token.text.lower() not in stopwords and
                            token.lemma_.lower() not in stopwords and
                            token.text.isalpha() and
                            token.lemma_.isalpha()):
                            
                            lemma = token.lemma_.lower()
                            word = token.text.lower()
                            
                            # Ajouter seulement si le mot est différent du lemme
                            if word != lemma:
                                self.lemma_to_words[lemma].add(word)
                
                except Exception as e:
                    print(f"Erreur lors du traitement d'un chunk: {e}")
                    continue
        
        print(f"Extraction terminée. {len(self.lemma_to_words)} lemmes collectés")
    
    def display_random_lemmas(self, n=50):
        """Affiche n lemmes au hasard avec leurs mots associés"""
        print(f"AFFICHAGE DE {n} LEMMES ALÉATOIRES AVEC LEURS MOTS")
        print("="*60)
        
        # Filtrer les lemmes qui ont au moins un mot associé différent
        valid_lemmas = {lemma: words for lemma, words in self.lemma_to_words.items() if len(words) > 0}
        
        if len(valid_lemmas) == 0:
            print("Aucun lemme avec des mots associés trouvé")
            return
        
        # Sélectionner aléatoirement n lemmes
        selected_lemmas = random.sample(list(valid_lemmas.keys()), min(n, len(valid_lemmas)))
        
        for i, lemma in enumerate(selected_lemmas, 1):
            associated_words = sorted(list(valid_lemmas[lemma]))
            words_str = ", ".join(associated_words)
            
            print(f"{i:2d}. {lemma:<15} -> {words_str}")
        
        print(f"\nTotal de lemmes disponibles : {len(valid_lemmas)}")
        print(f"Lemmes affichés : {len(selected_lemmas)}")

def main():
    """Analyse principale"""
    print("ANALYSE LEMMES-MOTS - DISCOURS POLITIQUES")
    print("="*50)
    
    try:
        mapper = LemmaWordMapper()
        
        # Chargement du modèle spaCy
        if not mapper.load_spacy_model():
            return
        
        # Chargement des données
        mapper.load_data()
        
        if len(mapper.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # Extraction du mapping lemmes-mots
        mapper.extract_lemma_word_mapping()
        
        # Affichage aléatoire
        mapper.display_random_lemmas(50)
        
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ANALYSE LEMMES-MOTS - DISCOURS POLITIQUES
Chargement du modèle spaCy français...
Modèle spaCy chargé avec succès
Chargement des données...
Nombre de discours chargés : 303
Extraction des lemmes et mots associés...
Stopwords chargés depuis /Users/charlielezin/Desktop/stopwords.txt: 707 mots
  Traitement: 50/303 discours
  Traitement: 100/303 discours
  Traitement: 150/303 discours
  Traitement: 200/303 discours
  Traitement: 250/303 discours
  Traitement: 300/303 discours
Extraction terminée. 3306 lemmes collectés
AFFICHAGE DE 50 LEMMES ALÉATOIRES AVEC LEURS MOTS
 1. accidenté       -> accidentés
 2. dresser         -> dressions, dressé, dressée
 3. mœur            -> mœurs
 4. fédéré          -> fédérés
 5. eur             -> eure
 6. remerciement    -> remerciements
 7. régir           -> régie
 8. identique       -> identiques
 9. ranger          -> rangés
10. gouverner       -> gouvernent, gouverné, gouvernée, gouvernés
11. disloquer       -> disloquait
12. mouvement       -> mouvem